# Lab Exercise 3: Matched Filters and L-ASK
<b>Importing packages we will need later in Python</b>

In [ ]:
# Import necessary libraries for signal processing and visualization

from scipy import signal                 # Provides signal processing tools
import numpy as np                        # Fundamental package for numerical computations
import matplotlib.pyplot as plt           # Plotting library for creating static, animated, and interactive visualizations
from scipy.special import erfc            # Contains the complementary error function and other special functions
import ipywidgets as widgets              # Enables the use of interactive HTML widgets in Jupyter notebooks
from ipywidgets import RadioButtons       # Widget for creating radio button groups
from IPython.display import display, clear_output  # Functions to display output and clear cell outputs
import scipy.signal
from scipy.signal import upfirdn, welch, decimate, convolve, firwin2, lfilter # Functions for resampling and convolution operations
from ipywidgets import Checkbox, Button, Output, VBox, HBox, Dropdown, Layout  # Widgets for interactive controls and layout management
import time                               # Time-related functions for handling timing and delays
from commpy.channels import awgn

# Confirmation message indicating successful import of libraries
print("Libraries added successfully!")

After studying <b><a href='https://helios.ntua.gr/mod/resource/view.php?id=44639'>Chapter 3</a></b> and, in particular, Example 3.2 of the notes, you are given the following Python function ask_errors() that emulates the generation and decoding of noisy L-ASK signals and calculates the number of false symbols

In [ ]:
# k is the number of bits per symbol, so L=2^k is the number of different pulses
# M is the number of the generated symbols (length of signal)
# nsamp is the number of samples per symbol (oversampling ratio)
# EbNo is the normalized signal to noise ratio,Eb/No, in db
def ask_errors(k, M, nsamp, EbNo):
    # L is the number of different amplitude levels (2^k)
    L = 2**k
    # Calculate SNR in dB adjusting for number of samples and bits per symbol
    SNR_db = EbNo - 10*np.log10(nsamp/2/k)
    # Convert SNR from dB to linear scale
    SNR = 10 ** (SNR_db * 0.1)
    # Generate a vector of random symbols with amplitudes {±1, ±3, ... ±(L-1)}
    x = 2*np.floor(L*np.random.rand(M)) - L + 1
    # Theoretical signal power
    P_x = (L**2-1) / 3
    # Measured signal power (for verification)
    Measured_x = np.sum(x*x)/len(x)

    # Generate oversampled signal y by repeating each symbol nsamp times
    y = []
    for i in range(len(x)):
        for j in range(nsamp):
            y.append(x[i])
    y = np.array(y)

    # Generate noise with zero mean and variance based on the signal power and SNR
    noise = np.random.normal(0, np.sqrt(Measured_x/SNR), len(y))
    # Add noise to the signal
    y_noisy = y + noise

    # Reshape the noisy signal for matched filtering
    y = np.reshape(y_noisy, (M, nsamp))
    # Create a matched filter with coefficients all ones
    matched = np.ones((nsamp, 1))
    # Apply matched filter to the signal
    z = np.matmul(y, matched)
    z = z / nsamp
    # Levels for decision making {±1, ±3, ... ±(L-1)}
    l = np.arange(-L+1, L, 2)

    # Decision making: for each symbol, find the closest amplitude level
    z = z[:, 0]
    for i in range(len(z)):
        differences = np.abs(l - z[i]) # Array of differences from the signal to the levels
        m = min(differences)
        [index], = np.where(differences == m)
        z[i] = l[index]
    
    # Count errors: compare the decided symbols with the original symbols
    errors = 0
    for i in range(len(z)):
        if x[i] != z[i]:
            errors += 1
    
    # Return the total number of symbol errors
    return errors

## Part 1: Simulation code, modifications and code verification
(a) Modify the code of ask_errors() so that the L elements of vector x get values from the set {±d/2, ±3d/2, ±5d/2...}, where the distance d of the points will be given as a parameter. Using the value d=5, verify by calculation and displaying a relevant histogram, that the elements of vector x indeed follow a uniform distribution. Use k=mod(nnnnn,2)+3, where nnnnn is the last 5-digit part of your academic id number.<br>
<b>Hint</b>: Generate at least 40,000 random symbols, and use the command hist(x,A) to calculate and display the histogram, where A is the vector of the L different values of these integers. Modify appropriately the code for the theoretical calculation of power and verify with Measured signal power. Similarly, modify l vector appropriately.

In [ ]:
# k is the number of bits per symbol, so L=2^k is the number of different pulses
# M is the number of the generated symbols (length of signal)
# nsamp is the number of samples per symbol (oversampling ratio)
# EbNo is the normalized signal to noise ratio,Eb/No, in db

def ask_errors_new(k, M, nsamp, EbNo):
# d distance between ASK pulses
    d=5

    return errors

(b) With parameter values of the function k=4, M=60000, nsamp=20, and EbNo=12, run ask_errors() and  plot the histogram of z with bins=200. Repeat the above for EbNo=16 and EbNo=20. What do you observe? Explain the differences in the three diagrams.

In [ ]:
k=4
M=6000
nsamp=20
EbNo=12
d=5
L = 2**k


# Plot the histogram of the x

# Plot the histogram of the z


(c) Answer the following questions:
<li>What do the following code lines calculate:</li>

````python
y = np.reshape(y_noisy, (M, nsamp)) 
z = z / nsamp 

````
<li>What type and dimension are the variables: <br>x, <br>y after code line y = np.array(y), <br>y after code line y = np.reshape(y_noisy, (M, nsamp)), matched, z and errors?</li>
(d) Explain the operation of the loop for Decision making of the code as a minimum distance detector for L-ASK.

## Part 2: Performance Curves (BER as a function of the signal-to-noise ratio)

Verify the curve of Figure 3.10 of the notes (for convenience repeated below) for L-ASK, with L=2^k, k=mod(nnnnn, 2)+3, where nnnnn is the last 5-digit part of your registration number. Plot the theoretical curve and superimpose the simulation results (discrete points), as in the figure. 'Read' from the 8-ASK curve and write in the submission file the BER values for Eb/No={8,14,18}db. Conversely, find the appropriate Eb/No value for an 8-ASK system with a transmission rate of R=1Mbps, so that the error rate does not exceed the value {10, 100, 1000} bps.

![lab3_1.png](lab3_1.png)

Plot the curves in two ways:

(a) With your own program, which (i) will plot the theoritical curve by using the following equation (3.33 equation of Chapter 3) with the approximation BER≈Pe/log2L, and (ii) will plot the experimental curve by calling the function ask_errors() for calculating the discrete points.


$
P_e = \frac{L-1}{L} \cdot \text{erfc}\left( \sqrt{\frac{3 \log_2 L  }{L^2 - 1} \cdot \frac{E_{b,av}}{N_o}} \right)
$


(where Pe is the false symbol probability)

Hint: (a) Call ask_errors(), once for each different value of EbNo, with a sufficiently high value of M (e.g., 20000) and calculate the error rate.


In [ ]:
k=
M=10000
nsamp=20

# ber_simulation = ask_errors(k, M, nsamp, EbNo) / M / np.log2(L)
# ber_theoretical = ...
# Use semilogy() function of matplotlib

(b) Using the following code that plots the theoritical BER curve as well as the experimental one of ASK modulation, using ask_errors() function as already implemented in the above cells. The L parameter of ASK is selected below the cell

In [ ]:
# Loading animation
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey */
                                     border-top: 12px solid #01cc97; /* Blue */
                                     border-radius: 50%;
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite;'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """
done = """
        <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
            <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>
        </div>
        """
loader_html3 = widgets.HTML(
  value=loading
)
timer_html3 = widgets.HTML(
    value="Elapsed time: - seconds"
)

# Define the modulation schemes and levels
modulation_schemes = {
    'ASK': ['2-ASK', '4-ASK', '8-ASK', '16-ASK', '32-ASK', '64-ASK']
}

# Dropdown for modulation schemes
modulation_dropdown = widgets.Dropdown(
    options=list(modulation_schemes.keys()),
    value='ASK',
    description='Modulation:'
)

# Dropdown for levels
levels_dropdown = widgets.Dropdown(
    description='Levels:'
)


def update_levels_dropdown(*args):
    levels_dropdown.options = modulation_schemes[modulation_dropdown.value]
    levels_dropdown.value = levels_dropdown.options[0]  # Set default value


modulation_dropdown.observe(update_levels_dropdown, 'value')
update_levels_dropdown()  # Initialize the levels dropdown

plot_output = widgets.Output()

# Give thw following parameters
M = 2000 # number of symbols for simulation
nsamp = 16 
EbN0_db = np.arange(1, 15, 2) # EbNo range
EbN0 = 10 ** (EbN0_db / 10) # EbNo linear value
EbNo_dB_theory = np.arange(0, 15, 1)  # Eb/No values in dB for theoretical
EbNo_theory = 10 ** (EbNo_dB_theory / 10)  # convert dB to linear for theoretical

def plot_selected_modulations(change):
    # Start the timer
    loader_html3.value = loading
    start_time = time.time()

    with plot_output:
        loader_html3.value = loading
        plot_output.clear_output(wait=True)
        plt.figure(figsize=(10, 7))
        
        modulation = modulation_dropdown.value
        level = levels_dropdown.value
        if level is None:
            return
        if modulation == 'MSK':
            L = 2
        else:
            L = int(level.split('-')[0])
        L1 = int(np.sqrt(L))
        k = int(np.log2(L))
        modulation_name = level
 # Code will use your ask_errors function  ################     
 ##########################################################
 # Change the ask_errors() function on the following line #
        ber = [ask_errors(k, M, nsamp, db) / M / np.log2(L) for db in EbN0_db]
        ber_theoretical = (((L - 1) / L) * scipy.special.erfc(np.sqrt(EbN0 * (3 * np.log2(L)) / (L**2 - 1)))) / k
        plt.semilogy(EbN0_db, ber_theoretical, linestyle='-', label=f'Theoretical {modulation_name}')
        plt.semilogy(EbN0_db, ber, 'o', label=f'Experimental {modulation_name}')
        
        plt.grid(True, which='both')
        plt.xlabel("Eb/N0 (dB)")
        plt.ylabel("Bit Error Rate")
        plt.legend()
        plt.title(f'{modulation_name} | Theoretical and Experimental BER')

        # Show elapsed time
        elapsed_time = time.time() - start_time
        timer_html3.value = f"Elapsed time: {elapsed_time:.2f} seconds"
        loader_html3.value = done

        plt.show()

# Define the plot button
plot_button = widgets.Button(description="Plot", button_style='primary')

# Attach the plot_selected_modulations function to the 'click' event of the button
plot_button.on_click(plot_selected_modulations)

inputs = widgets.VBox([modulation_dropdown, levels_dropdown])

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html3, timer_html3], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

inputs_ui = widgets.HBox([inputs, loader_timer_box], layout=Layout(align_items='center'))

ui = widgets.VBox([inputs_ui, plot_button]) 

def update_ui(change):
    inputs_ui.children = [inputs, loader_timer_box]
#inputs_ui.children = [inputs, loader_timer_box]

# Attach the update_ui function to the 'value' property of the modulation dropdown
modulation_dropdown.observe(update_ui, names='value')

# Call the update_ui function initially to set the correct UI state
update_ui(None)

# Setup the display layout
display(ui, plot_output)

# Display the initial plot
plot_selected_modulations(None)

## Part 4: Implementation with Convolution - Using Other Pulses
<b> Modify ask_errors() function code. Use the following commands </b>
````python
# Filter impulse response: orthogonal pulse of unit energy
h = np.ones(nsamp) / np.sqrt(nsamp)
    
# Upsample x by inserting zeros between samples. Note: 'x' should be a numpy array for direct indexing <br>
y_upsampled = np.zeros(M * nsamp)  # Preallocate upsampled signal array <br>
y_upsampled[::nsamp] = x  # Assign every nsamp-th sample to x, leaving zeros in between
    
# Convolution of the upsampled signal with the filter impulse response
y = np.convolve(y_upsampled, h, mode='full')[:M*nsamp]  # Trim the convolution tail

````
Also, the noisy signal, y_noisy, should be produced with the command:
````python
# Calculate signal power and convert SNR from dB to linear
signal_power = np.mean(y**2)
SNR = 10**(SNR_dB / 10)
    
# Calculate noise power to achieve desired SNR
noise_power = signal_power / SNR
    
# Generate noise with calculated power
noise = np.random.normal(0, np.sqrt(noise_power), y.shape)
    
# Add noise to signal
y_noisy = y + noise

````
and the matched filter to be implemented with convolution as:
````python
# Create the matched filter by reversing 'h'
matched = h[::-1]
   
# Convolve the noisy signal with the matched filter
yrx = np.convolve(y_noisy, matched, mode='full')
    
# Sample the result at the end of each symbol period
# Note: Python indexing starts at 0, so we adjust the start index accordingly
z = yrx[nsamp-1:M*nsamp:nsamp]

````
(the command that does reshape on the ynoisy signal must, of course, be removed here).

In [ ]:
# k is the number of bits per symbol, so L=2^k is the number of different pulses
# M is the number of the generated symbols (length of signal)
# nsamp is the number of samples per symbol (oversampling ratio)
# EbNo is the normalized signal to noise ratio,Eb/No, in db

def ask_errors_part4(k, M, nsamp, EbNo):
    
    return errors


<b>(a) Confirm that the modified ask_errors() code produces the same results</b>

In [ ]:
print('Number of symbol errors:',ask_errors_part4(4, 10000, 16, 18))
print('Number of symbol errors:',ask_errors(4, 10000, 16, 18))
print('Number of symbol errors:',ask_errors_new(4, 10000, 16, 18))

<b>(b) After executing the body of the modified function ask_errors() without adding noise (i.e., with ynoisy=y), plot a section of the signals x, y, and yrx. Write a brief explanation of the second (y) and third diagrams (yrx). </b>

In [ ]:
k = 3         # Number of bits per symbol (e.g., k=3 for 8-ASK)
M = 10000       # Total number of symbols to simulate
nsamp = 16    # Number of samples per symbol (oversampling factor)
L = 2 ** k  # Modulation order (number of signal levels)



<b>(c) Replace the rectangular pulse h with another one </b><br>
e.g., a cosine of one period: 
````python
h = np.cos(2 * np.pi * np.arange(1, nsamp + 1) / nsamp)
h = h / np.sqrt(np.sum(h ** 2))  # Normalize the pulse to have unit energy
````
Execute the simulation again and compare with the theoritical. What is the conclusion about the performance of the new L-ASK system? If we do not revert the matched filter (set matched = h), we will observe that we do not get correct results, especially for small values of nsamp (e.g., nsamp=8). Explain this, after observing the shape of the pulse h in the two cases (rectangular pulse and sine pulse), plotting it with stems (stem(h)). Repeat the experiment for nsamp=32 and 64.

In [ ]:
# k is the number of bits per symbol, so L=2^k is the number of different pulses
# M is the number of the generated symbols (length of signal)
# nsamp is the number of samples per symbol (oversampling ratio)
# EbNo is the normalized signal to noise ratio,Eb/No, in db
def ask_errors_part4c(k, M, nsamp, EbNo):

    return errors